In [1]:
import pandas as pd
import numpy as np
import time

from sklearn import linear_model, svm
from sklearn.linear_model import LogisticRegression

import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit,\
    cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,\
    roc_auc_score

import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


pd.set_option('display.max_columns',99)
pd.set_option('display.max_rows',300)

In [20]:
#READ CSVs

#xing
train_unique_orig = pd.read_csv('dataset_diabetes/data2/train_unique_cleaned_df.csv')
test_unique_orig = pd.read_csv('dataset_diabetes/data2/test_unique_cleaned_df.csv')

#15,20,25,30 whole and unique

train_unique_15_orig = pd.read_csv('dataset_diabetes/experiment/new/15/unique_train_cleaned_15.csv')
train_whole_15_orig = pd.read_csv('dataset_diabetes/experiment/new/15/whole_train_cleaned_15.csv')
test_unique_15_orig = pd.read_csv('dataset_diabetes/experiment/new/15/unique_test_cleaned_15.csv')

train_unique_20_orig = pd.read_csv('dataset_diabetes/experiment/new/20/unique_train_cleaned_20.csv')
train_whole_20_orig = pd.read_csv('dataset_diabetes/experiment/new/20/whole_train_cleaned_20.csv')
test_unique_20_orig = pd.read_csv('dataset_diabetes/experiment/new/20/unique_test_cleaned_20.csv')

train_unique_25_orig = pd.read_csv('dataset_diabetes/experiment/new/25/unique_train_cleaned_25.csv')
train_whole_25_orig = pd.read_csv('dataset_diabetes/experiment/new/25/whole_train_cleaned_25.csv')
test_unique_25_orig = pd.read_csv('dataset_diabetes/experiment/new/25/unique_test_cleaned_25.csv')

train_unique_30_orig = pd.read_csv('dataset_diabetes/experiment/new/30/unique_train_cleaned_30.csv')
train_whole_30_orig = pd.read_csv('dataset_diabetes/experiment/new/30/whole_train_cleaned_30.csv')
test_unique_30_orig = pd.read_csv('dataset_diabetes/experiment/new/30/unique_test_cleaned_30.csv')

In [21]:
#make copies

train_unique_xing = train_unique_orig.copy()
test_unique_xing = test_unique_orig.copy()


train_unique_15 = train_unique_15_orig.copy()
test_unique_15 = test_unique_15_orig.copy()
train_whole_15 = train_whole_15_orig.copy()

train_unique_20 = train_unique_20_orig.copy()
test_unique_20 = test_unique_20_orig.copy()
train_whole_20 = train_whole_20_orig.copy()

train_unique_25 = train_unique_25_orig.copy()
test_unique_25 = test_unique_25_orig.copy()
train_whole_25 = train_whole_25_orig.copy()

train_unique_30 = train_unique_30_orig.copy()
test_unique_30 = test_unique_30_orig.copy()
train_whole_30 = train_whole_30_orig.copy()

In [29]:
#train_unique_30.sample(10)

In [22]:
def prep_dfs(split,train_unique,train_whole,test_unique):
    
    train_unique_y = train_unique['readmitted']
    unique_encounter_patient = train_unique[['encounter_id','patient_nbr']]
    train_unique.drop(['readmitted','encounter_id','patient_nbr'], inplace = True, axis = 1)
    
    train_whole_y = train_whole['readmitted']
    whole_encounter_patient = train_whole[['encounter_id','patient_nbr']]
    train_whole.drop(['readmitted','encounter_id','patient_nbr'], inplace = True, axis = 1)
    
    test_unique_y = test_unique['readmitted']
    test_unique.drop(['readmitted','encounter_id','patient_nbr'], inplace = True, axis = 1)
    
    print('{split} percent X_train_unique shape is {shape}'.format(split = split, shape = train_unique.shape))
    print('{split} percent y_train_unique shape is {shape}'.format(split = split, shape = train_unique_y.shape))
    print('{split} percent X_train_whole shape is {shape}'.format(split = split, shape = train_whole.shape))
    print('{split} percent y_train_whole shape is {shape}'.format(split = split, shape = train_whole_y.shape))
    print('{split} percent X_test_unique shape is {shape}'.format(split = split, shape = test_unique.shape))
    print('{split} percent y_test_unique shape is {shape}'.format(split = split, shape = test_unique_y.shape))
    print(' ')
    
    return train_unique, train_unique_y, unique_encounter_patient, train_whole, train_whole_y,\
        whole_encounter_patient,test_unique,test_unique_y

In [23]:
#create dataframes for models
X_train_unique_15, y_train_unique_15, unique_15_encounter_patient, X_train_whole_15,\
y_train_whole_15, whole_15_encounter_patient,X_test_unique_15,y_test_unique_15 = prep_dfs(15,train_unique_15,\
                                                                       train_whole_15,\
                                                                       test_unique_15)

X_train_unique_20, y_train_unique_20, unique_20_encounter_patient, X_train_whole_20,\
y_train_whole_20, whole_20_encounter_patient,X_test_unique_20,y_test_unique_20 = prep_dfs(20,train_unique_20,\
                                                                       train_whole_20,\
                                                                       test_unique_20)

X_train_unique_25, y_train_unique_25, unique_25_encounter_patient, X_train_whole_25,\
y_train_whole_25, whole_25_encounter_patient,X_test_unique_25,y_test_unique_25 = prep_dfs(25,train_unique_25,\
                                                                        train_whole_25,\
                                                                        test_unique_25)

X_train_unique_30, y_train_unique_30, unique_30_encounter_patient, X_train_whole_30,\
y_train_whole_30, whole_30_encounter_patient,X_test_unique_30,y_test_unique_30 = prep_dfs(30,train_unique_30,\
                                                                       train_whole_30,\
                                                                       test_unique_30)

15 percent X_train_unique shape is (61674, 91)
15 percent y_train_unique shape is (61674,)
15 percent X_train_whole shape is (84422, 91)
15 percent y_train_whole shape is (84422,)
15 percent X_test_unique shape is (13835, 91)
15 percent y_test_unique shape is (13835,)
 
20 percent X_train_unique shape is (58793, 91)
20 percent y_train_unique shape is (58793,)
20 percent X_train_whole shape is (79456, 91)
20 percent y_train_whole shape is (79456,)
20 percent X_test_unique shape is (18019, 91)
20 percent y_test_unique shape is (18019,)
 
25 percent X_train_unique shape is (55815, 91)
25 percent y_train_unique shape is (55815,)
25 percent X_train_whole shape is (74490, 91)
25 percent y_train_whole shape is (74490,)
25 percent X_test_unique shape is (22049, 91)
25 percent y_test_unique shape is (22049,)
 
30 percent X_train_unique shape is (52825, 91)
30 percent y_train_unique shape is (52825,)
30 percent X_train_whole shape is (69524, 91)
30 percent y_train_whole shape is (69524,)
30 perc

In [24]:
#dictionary to store all the different dataframes with different splits
versions = {'X_train':{'15w':X_train_whole_15,'15u':X_train_unique_15,\
                       '20w':X_train_whole_20,'20u':X_train_unique_20,\
                       '25w':X_train_whole_25,'25u':X_train_unique_25,\
                       '30w':X_train_whole_30,'30u':X_train_unique_30},\
            'y_train':{'15w':y_train_whole_15,'15u':y_train_unique_15,\
                       '20w':y_train_whole_20,'20u':y_train_unique_20,\
                       '25w':y_train_whole_25,'25u':y_train_unique_25,\
                       '30w':y_train_whole_30,'30u':y_train_unique_30},
            'X_test':{ '15u':X_test_unique_15,\
                       '20u':X_test_unique_20,\
                       '25u':X_test_unique_25,\
                       '30u':X_test_unique_30},\
            'y_test':{ '15u':y_test_unique_15,\
                       '20u':y_test_unique_20,\
                       '25u':y_test_unique_25,\
                       '30u':y_test_unique_30}}

In [232]:
#versions['X_train']['15w'].head()

In [33]:
#versions['X_test']['15u'].head()

In [34]:
#versions['y_test']['15u'].head()

In [25]:
#declare which version to run in model below:

def choose_version(data_split,selection):

    X_train = versions['X_train'][str(data_split)+selection] 
    y_train = versions['y_train'][str(data_split)+selection]
    X_test = versions['X_test'][str(data_split)+'u']
    y_test = versions['y_test'][str(data_split)+'u']
    
    return X_train, y_train, X_test, y_test

In [26]:
#++++++++++++++++RUN THIS CELL TO SET WHICH SPLIT VERSION:++++++++++++++++++++

X_train, y_train, X_test, y_test = choose_version(30,'w')

# X_train, y_train, X_test, y_test = choose_version(25,'u')
# X_train, y_train, X_test, y_test = choose_version(30,'w')

#### LightGBM

In [27]:
def objective_function(params):
    clf = lgb.LGBMClassifier(**params)
    skf = ms.StratifiedKFold(n_splits=10, shuffle=True, random_state=99)
    score = cross_val_score(clf, X_train, y_train, scoring = 'roc_auc', cv=skf, n_jobs=1)
    return {'loss': 1-score.mean(), 'status': STATUS_OK}     

In [28]:
num_eval = 100

param_hyperopt= {
                'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(2)),
                'max_depth': scope.int(hp.quniform('max_depth', 5, 19, 1)), #'max_depth', 5, 19, 1
                'n_estimators': scope.int(hp.quniform('n_estimators', 5, 55, 1)), #'n_estimators', 5, 55, 1
                'num_leaves': scope.int(hp.quniform('num_leaves', 5, 38, 1)),
                'boosting_type': hp.choice('boosting_type', ['gbdt','dart','goss']), #check lightgbm for types
                'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1.0),
                'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.5), #ridge
                'drop_rate': hp.uniform('drop_rate', 0.0, 1.0),
                'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.1),
                'max_bin': scope.int(hp.quniform('max_bin', 255, 265, 1)),
                'min_child_weight' : scope.int(hp.quniform('min_child_weight', 8, 20, 1))
                #'scale_pos_weight': hp.uniform('scale_pos_weight', 1.0,12.0),
                }

trials = Trials()
best_param = fmin(objective_function, 
                  param_hyperopt, 
                  algo=tpe.suggest, 
                  max_evals=num_eval, 
                  trials=trials,
                  rstate= np.random.RandomState(1))
loss = [x['result']['loss'] for x in trials.trials]
best_param_values = [x for x in best_param.values()]

100%|██████████| 100/100 [06:55<00:00,  4.48s/it, best loss: 0.332567383379216] 


In [274]:
round(11.34532,2)

11.35

In [29]:
best_param

{'boosting_type': 0,
 'colsample_bytree': 0.2566946790660347,
 'drop_rate': 0.9195674755017068,
 'learning_rate': 0.11449394808202824,
 'max_bin': 261.0,
 'max_depth': 12.0,
 'min_child_weight': 19.0,
 'n_estimators': 48.0,
 'num_leaves': 28.0,
 'reg_alpha': 0.9963376427069566,
 'reg_lambda': 1.4070988716151476}

In [184]:
#best_param

{'boosting_type': 0,
 'colsample_by_tree': 0.5402954683529296,
 'drop_rate': 0.7736268944483489,
 'learning_rate': 0.02190745895434768,
 'max_depth': 27.0,
 'n_estimators': 54.0,
 'num_leaves': 48.0,
 'reg_lambda': 0.6542818431578171}

In [30]:
if best_param_values[0] == 0:
    boosting_type = 'gbdt'

elif best_param_values[0] == 1:
    boosting_type = 'dart'

else:
    #best_param_values[0] == 2
    boosting_type = 'goss'    
    
# else:
#     boosting_type = 'goss'
        
    
clf_best = lgb.LGBMClassifier(boosting_type=boosting_type, #'dart'
                              colsample_bytree = best_param['colsample_bytree'],
                              drop_rate = best_param['drop_rate'],
                              learning_rate = best_param['learning_rate'],
                              max_bin = int(best_param['max_bin']),
                              max_depth = int(best_param['max_depth']),
                              n_estimators = int(best_param['n_estimators']),
                              num_leaves = int(best_param['num_leaves']),
                              reg_alpha = best_param['reg_alpha'],
                              reg_lambda = best_param['reg_lambda'],
                              objective = 'binary',
                              #scale_pos_weight = best_param['scale_pos_weight'],
                              #bagging_fraction = 
                              )
                                  
clf_best.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.2566946790660347,
               drop_rate=0.9195674755017068, importance_type='split',
               learning_rate=0.11449394808202824, max_bin=261, max_depth=12,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=48, n_jobs=-1, num_leaves=28, objective='binary',
               random_state=None, reg_alpha=0.9963376427069566,
               reg_lambda=1.4070988716151476, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [31]:
print("")
print("##### Results")
print("Score best parameters: ", min(loss)*-1)
print("Best parameters: ", best_param)
print("Test Score: ", clf_best.score(X_test, y_test))
#print("Time elapsed: ", time.time() - start)
print("Parameter combinations evaluated: ", num_eval)
auc = roc_auc_score(y_test, clf_best.predict_proba(X_test)[:,1])
auct = roc_auc_score(y_train, clf_best.predict_proba(X_train)[:,1])
print('AUC train is: ',auct)
print('AUC test is: ',auc)


##### Results
Score best parameters:  -0.332567383379216
Best parameters:  {'boosting_type': 0, 'colsample_bytree': 0.2566946790660347, 'drop_rate': 0.9195674755017068, 'learning_rate': 0.11449394808202824, 'max_bin': 261.0, 'max_depth': 12.0, 'min_child_weight': 19.0, 'n_estimators': 48.0, 'num_leaves': 28.0, 'reg_alpha': 0.9963376427069566, 'reg_lambda': 1.4070988716151476}
Test Score:  0.9041772836074434
Parameter combinations evaluated:  100
AUC train is:  0.6973692029358518
AUC test is:  0.6550757972867017


In [33]:
clf_best.plot_importance

AttributeError: 'LGBMClassifier' object has no attribute 'plot_importance'

In [131]:
results = {'30u':0.6535859962867702, '30w':0.6545053440555348, '30w':0.6559087210856963}